In [2]:
import collections
import csv
import math
import json
import os
import gzip
import pprint
import sys

import requests
import pubchempy

In [5]:
# LINCS API variables
# http://api.lincscloud.org/
api_version = 'a2'
service = 'pertinfo'
api_url = 'http://api.lincscloud.org/{}/{}'.format(api_version, service)
with open(os.path.join('private', 'apikey.txt')) as read_file:
    api_key = read_file.read().rstrip()
block_size = 100 # max is 1000

In [ ]:
def json_from_url(base, params):
    """Parse and return a json file obtained from the specified url."""
    response = requests.get(base, params=params)
    return response.json()

def query_links_api(query, verbose=False):
    """ """
    url_data = {'q': query, 'l': block_size, 'sk': 0, 'user_key': api_key, 'c': 'true'}
    
    num_docs = json_from_url(api_url, url_data)['count']
    num_blocks = int(math.ceil(float(num_docs) / block_size))
    del url_data['c']
    if verbose:
        print '{} results: splitting query into {} chunks of {}.'.format(num_docs, num_blocks, block_size)
    
    results = list()
    for i in range(num_blocks):
        if verbose:
            print 'Chunk {}/{}'.format(i + 1, num_blocks)
        url_data['sk'] =  i * block_size
        results += json_from_url(api_url, url_data)
    
    return results

In [ ]:
query = '{"pert_type":"trt_cp"}'
compounds = query_links_api(query)

In [ ]:
path = os.path.join('data', 'lincs_small_molecules.json.gz')
with gzip.open(path, 'w') as write_file:
    json.dump(compounds, write_file)